In [1]:
import numpy as np
import tensorflow as tf
from utils import *
from tensorflow.contrib import learn
from tensorflow.contrib import layers
from tensorflow.contrib.learn import *
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib

tf.logging.set_verbosity(tf.logging.INFO)

## Input function for training

- Get image data from the given directory 
- Put the data into TensorFlow Queue
- Return (features, label)
    - features: a Tensor with shape (batch_size, height, width, channels) 
    - label: a Tensor with shape (batch_size)

In [2]:
train_input_fn = read_img(data_dir='data/train', batch_size=32, shuffle=True)

In [3]:
train_input_fn()

(<tf.Tensor 'features:0' shape=(?, 256, 256, 3) dtype=uint8>,
 <tf.Tensor 'label:0' shape=(?,) dtype=int32>)

### Define CNN model function

In [4]:
def cnn_model_fn(features, labels, mode, params):
    """
    Model function for CNN
    :param features: images features with shape (batch_size, height, width, channels)
    :param labels: images category with shape (batch_size)
    :param mode: Specifies if this training, evaluation or
                 prediction. See `model_fn_lib.ModeKey`
    :param params: dict of hyperparameters
    :return: predictions, loss, train_op, Optional(eval_op). See `model_fn_lib.ModelFnOps`
    """
    
    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
        inputs=features,
        filters=32,
        kernel_size=[3, 3],
        padding="same",
        activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    
    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[3, 3],
        padding="same",
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Convolutional Layer #3 and Pooling Layer #3
    conv3 = tf.layers.conv2d(
        inputs=pool2,
        filters=64,
        kernel_size=[3, 3],
        padding="same",
        activation=tf.nn.relu)
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)
    
    # Dense Layer
    pool_flat = tf.reshape(pool3, [-1, 32 * 32 * 64])
    dense = tf.layers.dense(inputs=pool_flat, units=512, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=params['drop_out_rate']
                                , training=mode == learn.ModeKeys.TRAIN)
    
    # Logits Layer, a final layer before applying softmax
    logits = tf.layers.dense(inputs=dropout, units=17)
    
    loss = None
    train_op = None
    
    # Calculate Loss (for both TRAIN and EVAL modes)
    if mode != learn.ModeKeys.INFER:
        onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=17, name="onehot")
        #cross entropy loss
        loss = tf.losses.softmax_cross_entropy(
            onehot_labels=onehot_labels, logits=logits)
        
    # Configure the Training Op (for TRAIN mode)
    if mode == learn.ModeKeys.TRAIN:
        train_op = tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.contrib.framework.get_global_step(),
            optimizer=tf.train.AdamOptimizer,
            learning_rate=params['learning_rate'],
            summaries=[
                "learning_rate",
                "loss",
                "gradients",
                "gradient_norm",
            ])
    
    # Generate Predictions
    predictions = {
        "classes": tf.argmax(
            input=logits, axis=1),
        "probabilities": tf.nn.softmax(
            logits, name="softmax_tensor")
    }
    
    # Return a ModelFnOps object
    return model_fn_lib.ModelFnOps(mode=mode, 
                                   predictions=predictions, 
                                   loss=loss, 
                                   train_op=train_op,
                                   eval_metric_ops={'dense': dense})

### Define Data Preprocessing

In [5]:
def feature_engineering_fn(features, labels):
    """
    feature_engineering_fn: Feature engineering function. Takes features and
                              labels which are the output of `input_fn` and
                              returns features and labels which will be fed
                              into `model_fn`
    """
    
    features = tf.to_float(features)
    
    # Preprocessing or Data Augmentation
    # tf.image implements most of the standard image augmentation

    # Example
    # Subtract off the mean and divide by the variance of the pixels.
    features = tf.map_fn(tf.image.per_image_standardization, features)
    
    return features, labels

### Instantiate an Estimator

In [6]:
run_config = RunConfig(save_summary_steps=10, keep_checkpoint_max=2, save_checkpoints_secs=30)
#drop_out_rate = 0.2, learning_rate = 0.0001
params = {'drop_out_rate': 0.2, 'learning_rate': 0.0001}
#use "model/plain_cnn" as model_dir
cnn_classifier = learn.Estimator(model_fn=cnn_model_fn, model_dir="_model/plain_cnn",
        config=run_config,
        feature_engineering_fn=feature_engineering_fn, params=params)

Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
Instructions for updating:
Please replace uses of any Estimator from tf.contrib.learn with an Estimator from tf.estimator.*
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002267A883F98>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 10, '_save_checkpoints_secs': 30, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 2, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '_model/plain_cnn'}


In [7]:
# Configure the accuracy metric for evaluation
metrics = {
    "accuracy":
        learn.MetricSpec(
            metric_fn=tf.metrics.accuracy, prediction_key="classes")
}

Instructions for updating:
Use tf.estimator.EstimatorSpec.eval_metric_ops.


### Define input function for validation monitor and Instantiate a Validation Monitor

In [8]:
#validation data is in 'data/validate' folder, batch size = 128
validate_input_fn = read_img(data_dir='data/validate', batch_size=128, shuffle=True)
validation_monitor = monitors.ValidationMonitor(input_fn=validate_input_fn,
                                                eval_steps=10,
                                                every_n_steps=50,
                                                metrics=metrics,
                                                name='validation')

Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.


### Start training the model

In [9]:
#use validation monitor defined above to evaluate model every 50 steps
cnn_classifier.fit(input_fn=train_input_fn, steps=300, monitors=[validation_monitor])

Instructions for updating:
Please switch to tf.train.get_global_step
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.EstimatorSpec. You can use the `estimator_spec` method to create an equivalent one.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from _model/plain_cnn\model.ckpt-610
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 610 into _model/plain_cnn\model.ckpt.
INFO:tensorflow:Starting evaluation at 2018-08-29-10:46:11
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from _model/plain_cnn\model.ckpt-610
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/10]
INFO:tensorflow:Evaluation [2/10]
INFO:tensorflow:Evaluation [3/10]
INFO:tensorflow:Evaluation [4/10]
INFO:tensorflow:Evaluation [5/10]
INFO:tensorflow:Evaluat

INFO:tensorflow:Saving checkpoints for 794 into _model/plain_cnn\model.ckpt.
INFO:tensorflow:Saving checkpoints for 802 into _model/plain_cnn\model.ckpt.
INFO:tensorflow:Saving checkpoints for 811 into _model/plain_cnn\model.ckpt.
INFO:tensorflow:global_step/sec: 0.197071
INFO:tensorflow:Starting evaluation at 2018-08-29-11:03:21
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from _model/plain_cnn\model.ckpt-811
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/10]
INFO:tensorflow:Evaluation [2/10]
INFO:tensorflow:Evaluation [3/10]
INFO:tensorflow:Evaluation [4/10]
INFO:tensorflow:Evaluation [5/10]
INFO:tensorflow:Evaluation [6/10]
INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Finished evaluation at 2018-08-29-11:04:22
INFO:tensorflow:Saving dict for global step 811: accuracy = 0.15625, Type 

Estimator(params={'drop_out_rate': 0.2, 'learning_rate': 0.0001})

### Final evaluation on unseen test data set

In [10]:
#test data is in 'data/test' folder, batch size = 512
test_input_fn = read_img(data_dir='data/test', batch_size=256, shuffle=False)
#steps = 1
eval_results = cnn_classifier.evaluate(input_fn=test_input_fn, metrics=metrics, steps=1)

INFO:tensorflow:Starting evaluation at 2018-08-29-11:14:36
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from _model/plain_cnn\model.ckpt-910
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2018-08-29-11:15:05
INFO:tensorflow:Saving dict for global step 910: accuracy = 0.17578125, Type of dense = <class 'numpy.ndarray'>, global_step = 910, loss = 11023.611
INFO:tensorflow:Summary for np.ndarray is not visible in Tensorboard by default. Consider using a Tensorboard plugin for visualization (see https://github.com/tensorflow/tensorboard-plugin-example/blob/master/README.md for more information).


In [11]:
np.save(os.getcwd() + '/embedding.npy', eval_results['dense'])